### Imports

In [100]:
from mne.io import read_raw_edf
import numpy as np
import os
import pandas as pd

### CSV Creation

In [101]:
main_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0'

patient_data = pd.DataFrame(columns="start,end".split(','))

,start,end


In [102]:
with open(f"{main_path}/RECORDS") as record_file:
    records = record_file.read().split()
    
with open(f"{main_path}/RECORDS-WITH-SEIZURES") as record_file:
    records_s = record_file.read().split()

for idx, count in enumerate(records):
    records[idx] = count[6:]

for idx, count in enumerate(records_s):
    records_s[idx] = count[6:]

In [103]:
''' Patient 01 '''

patient = 'chb01'

# Create lists of seizure start/end times paired with filenames
filenames = []
starts = list(-1 * np.ones(shape=46, dtype=int))
ends = list(-1 * np.ones(shape=46, dtype=int))

for n in list(np.linspace(start=1, stop=46, num=46, dtype=int)):
    if n == 3: starts[n-1], ends[n-1] = 2996, 3036
    elif n == 4: starts[n-1], ends[n-1] = 1467, 1494
    elif n == 15: starts[n-1], ends[n-1] = 1732, 1772
    elif n == 16: starts[n-1], ends[n-1] = 1015, 1066
    elif n == 18: starts[n-1], ends[n-1] = 1720, 1810
    elif n == 21: starts[n-1], ends[n-1] = 327, 420
    elif n == 26: starts[n-1], ends[n-1] = 1862, 1963
    
    if n <= 9: file_num = "0" + str(n)
    else: file_num = str(n)
    filenames.append(patient + f"_{file_num}.edf")

# Save to DataFrame
temp_df = pd.DataFrame(columns = patient_data.keys(), index=filenames)
temp_df["start"], temp_df["end"] = starts, ends
patient_data = pd.concat([patient_data, temp_df])

temp_df.head()

,start,end
chb01_01.edf,-1,-1
chb01_02.edf,-1,-1
chb01_03.edf,2996,3036
chb01_04.edf,1467,1494
chb01_05.edf,-1,-1


In [104]:
''' Patient 02 '''

patient = 'chb02'

# Create lists of seizure start/end times paired with filenames
filenames = []
starts = list(-1 * np.ones(shape=35, dtype=int))
ends = list(-1 * np.ones(shape=35, dtype=int))

for n in list(np.linspace(start=1, stop=35, num=35, dtype=int)):
    if n == 16: starts[n-1], ends[n-1] = 130, 212
    elif n == 19: starts[n-1], ends[n-1] = 3369, 3378
    
    if n <= 9: file_num = "0" + str(n)
    else: file_num = str(n)
    filenames.append(patient + f"_{file_num}.edf")

# Add file 16+
filenames.append(patient + f"_16+.edf")
starts.append(2972)
ends.append(3053)

# Save to DataFrame
temp_df = pd.DataFrame(columns = patient_data.keys(), index=filenames)
temp_df["start"], temp_df["end"] = starts, ends
temp_df.sort_index(axis='index', inplace=True)

patient_data = pd.concat([patient_data, temp_df])

temp_df.head()

,start,end
chb02_01.edf,-1,-1
chb02_02.edf,-1,-1
chb02_03.edf,-1,-1
chb02_04.edf,-1,-1
chb02_05.edf,-1,-1


In [105]:
''' Patient 03 '''

patient = 'chb03'

# Create lists of seizure start/end times paired with filenames
filenames = []
starts = list(-1 * np.ones(shape=38, dtype=int))
ends = list(-1 * np.ones(shape=38, dtype=int))

for n in list(np.linspace(start=1, stop=38, num=38, dtype=int)):
    if n == 1: starts[n-1], ends[n-1] = 362, 414
    elif n == 2: starts[n-1], ends[n-1] = 731, 796
    elif n == 3: starts[n-1], ends[n-1] = 432, 501
    elif n == 4: starts[n-1], ends[n-1] = 2162, 2214
    elif n == 34: starts[n-1], ends[n-1] = 1982, 2029
    elif n == 35: starts[n-1], ends[n-1] = 2592, 2656
    elif n == 36: starts[n-1], ends[n-1] = 1725, 1778
    
    if n <= 9: file_num = "0" + str(n)
    else: file_num = str(n)
    filenames.append(patient + f"_{file_num}.edf")
    
# Save to DataFrame
temp_df = pd.DataFrame(columns = patient_data.keys(), index=filenames)
temp_df["start"], temp_df["end"] = starts, ends
temp_df.sort_index(axis='index', inplace=True)

patient_data = pd.concat([patient_data, temp_df])

temp_df.head()

,start,end
chb03_01.edf,362,414
chb03_02.edf,731,796
chb03_03.edf,432,501
chb03_04.edf,2162,2214
chb03_05.edf,-1,-1


In [106]:
''' Patient 04 '''

patient = 'chb04'

# Create lists of seizure start/end times paired with filenames
filenames = []
starts = list(-1 * np.ones(shape=43, dtype=int))
ends = list(-1 * np.ones(shape=43, dtype=int))

for n in list(np.linspace(start=1, stop=43, num=43, dtype=int)):
    if n == 5: starts[n-1], ends[n-1] = 7804, 7853
    elif n == 8: starts[n-1], ends[n-1] = 6646, 6657
    elif n == 28: starts[n-1], ends[n-1] = 1679, 1781
    
    if n <= 9: file_num = "0" + str(n)
    else: file_num = str(n)
    filenames.append(patient + f"_{file_num}.edf")

# Add second seizure from file 28
filenames.append(patient + "_28.edf")
starts.append(3782); ends.append(3898)

# Save to DataFrame
temp_df = pd.DataFrame(columns = patient_data.keys(), index=filenames)
temp_df["start"], temp_df["end"] = starts, ends
temp_df.sort_index(axis='index', inplace=True)

patient_data = pd.concat([patient_data, temp_df])

temp_df.head()

,start,end
chb04_01.edf,-1,-1
chb04_02.edf,-1,-1
chb04_03.edf,-1,-1
chb04_04.edf,-1,-1
chb04_05.edf,7804,7853


In [107]:
''' Patient 05 '''

patient = 'chb05'

# Create lists of seizure start/end times paired with filenames
filenames = []
starts = list(-1 * np.ones(shape=39, dtype=int))
ends = list(-1 * np.ones(shape=39, dtype=int))

for n in list(np.linspace(start=1, stop=39, num=39, dtype=int)):
    if n == 6: starts[n-1], ends[n-1] = 417, 532
    elif n == 13: starts[n-1], ends[n-1] = 1086, 1196
    elif n == 16: starts[n-1], ends[n-1] = 2317, 2413
    elif n == 17: starts[n-1], ends[n-1] = 2451, 2571
    elif n == 22: starts[n-1], ends[n-1] = 2348, 2465
    
    if n <= 9: file_num = "0" + str(n)
    else: file_num = str(n)
    filenames.append(patient + f"_{file_num}.edf")

# Save to DataFrame
temp_df = pd.DataFrame(columns = patient_data.keys(), index=filenames)
temp_df["start"], temp_df["end"] = starts, ends
temp_df.sort_index(axis='index', inplace=True)

patient_data = pd.concat([patient_data, temp_df])

temp_df.head()

,start,end
chb05_01.edf,-1,-1
chb05_02.edf,-1,-1
chb05_03.edf,-1,-1
chb05_04.edf,-1,-1
chb05_05.edf,-1,-1


In [108]:
''' Patient 06 '''

patient = 'chb06'

# Create lists of seizure start/end times paired with filenames
filenames = []
starts = list(-1 * np.ones(shape=24, dtype=int))
ends = list(-1 * np.ones(shape=24, dtype=int))

for n in list(np.linspace(start=1, stop=24, num=24, dtype=int)):
    if n == 1: starts[n-1], ends[n-1] = 1724, 1738
    elif n == 4: starts[n-1], ends[n-1] = 327, 347
    elif n == 9: starts[n-1], ends[n-1] = 12500, 12516
    elif n == 10: starts[n-1], ends[n-1] = 10833, 10845
    elif n == 13: starts[n-1], ends[n-1] = 506, 519
    elif n == 18: starts[n-1], ends[n-1] = 7799, 7811
    elif n == 24: starts[n-1], ends[n-1] = 9387, 9403
    
    if n <= 9: file_num = "0" + str(n)
    else: file_num = str(n)
    filenames.append(patient + f"_{file_num}.edf")

# Add additional seizures from file 01
filenames.append(patient + "_01.edf")
starts.append(7461); ends.append(7476)

filenames.append(patient + "_01.edf")
starts.append(13525); ends.append(13540)

# Add additional seizure from file 04
filenames.append(patient + "_04.edf")
starts.append(6211); ends.append(6231)

# Save to DataFrame
temp_df = pd.DataFrame(columns = patient_data.keys(), index=filenames)
temp_df["start"], temp_df["end"] = starts, ends
temp_df.sort_index(axis='index', inplace=True)

patient_data = pd.concat([patient_data, temp_df])

In [109]:
''' Patient 07 '''

patient = 'chb07'

# Create lists of seizure start/end times paired with filenames
filenames = []
starts = list(-1 * np.ones(shape=19, dtype=int))
ends = list(-1 * np.ones(shape=19, dtype=int))

for n in list(np.linspace(start=1, stop=19, num=19, dtype=int)):
    if n == 12: starts[n-1], ends[n-1] = 4920, 5006
    elif n == 13: starts[n-1], ends[n-1] = 3285, 3381
    elif n == 19: starts[n-1], ends[n-1] = 13688, 13831
    
    if n <= 9: file_num = "0" + str(n)
    else: file_num = str(n)
    filenames.append(patient + f"_{file_num}.edf")

# Save to DataFrame
temp_df = pd.DataFrame(columns = patient_data.keys(), index=filenames)
temp_df["start"], temp_df["end"] = starts, ends
temp_df.sort_index(axis='index', inplace=True)
temp_df.head()

patient_data = pd.concat([patient_data, temp_df])

In [110]:
''' Patient 08 '''

patient = 'chb08'

# Create lists of seizure start/end times paired with filenames
filenames = []
starts = list(-1 * np.ones(shape=29, dtype=int))
ends = list(-1 * np.ones(shape=29, dtype=int))

for n in list(np.linspace(start=1, stop=29, num=29, dtype=int)):
    if n == 2: starts[n-1], ends[n-1] = 2670, 2841
    elif n == 5: starts[n-1], ends[n-1] = 2856, 3046
    elif n == 11: starts[n-1], ends[n-1] = 2988, 3122
    elif n == 13: starts[n-1], ends[n-1] = 2417, 2577
    elif n == 21: starts[n-1], ends[n-1] = 2083, 2347
    
    if n <= 9: file_num = "0" + str(n)
    else: file_num = str(n)
    filenames.append(patient + f"_{file_num}.edf")

# Save to DataFrame
temp_df = pd.DataFrame(columns = patient_data.keys(), index=filenames)
temp_df["start"], temp_df["end"] = starts, ends
temp_df.sort_index(axis='index', inplace=True)

temp_df.head()

patient_data = pd.concat([patient_data, temp_df])

In [118]:
# Delete rows with files that don't have seizures
for filename in patient_data.index:
    if filename not in (records_s):
        patient_data.drop(index=filename, inplace=True)

patient_data.rename_axis("filename", axis="index", inplace=True)
patient_data.head()

,start,end
filename,,
chb01_03.edf,2996,3036
chb01_04.edf,1467,1494
chb01_15.edf,1732,1772
chb01_16.edf,1015,1066
chb01_18.edf,1720,1810


In [119]:
patient_data.to_csv("patient_data.csv")

### Data CSV Generation

In [2]:
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_03' + '.edf', preload=False, verbose='ERROR')
ch_names = file.ch_names
fs = int(file.info['sfreq'])

In [3]:
hour3 = (np.arange(2996, 3037, step=1/fs) * int(fs)).astype(int)
hour4 = (np.arange(1467, 1495, step=1/fs) * int(fs)).astype(int)
hour15 = (np.arange(1732, 1773, step=1/fs) * int(fs)).astype(int)
hour16 = (np.arange(1015, 1067, step=1/fs) * int(fs)).astype(int)
hour18 = (np.arange(1720, 1811, step=1/fs) * int(fs)).astype(int)
hour21 = (np.arange(327, 421, step=1/fs) * int(fs)).astype(int)
hour26 = (np.arange(1862, 1964, step=1/fs) * int(fs)).astype(int)

In [4]:
num_true = len(hour3) + len(hour4) + len(hour15) + len(hour16) + len(hour18) + len(hour21) + len(hour26)
num_true

114944

In [5]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_03' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (0 – Normal, 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 2996 <= (i / fs) < 3037 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :])
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index)

# Undersample Class 0 samples via random selection to create balanced class ratio
time = pd.Series([i for i in range(3600) if i < 2996 or i >= 3037])
time = time.sample(n=int(len(x_temp2)/fs), replace=False).sort_values().multiply(fs)
for t in time:
    x_temp2 = pd.concat([x_temp2, x_temp1.iloc[t:t+256, :]])

x_data = x_temp2.copy().reset_index(drop=True)
print(x_data.shape)

(20992, 24)


In [6]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_04' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1467 <= (i / fs) < 1495 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
time = pd.Series([i for i in range(3600) if i < 1467 or i >= 1495])
time = time.sample(n=int(len(x_temp2)/fs), replace=False).sort_values().multiply(fs)
for t in time:
    x_temp2 = pd.concat([x_temp2, x_temp1.iloc[t:t+256, :]])

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(35328, 24)


In [7]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_15' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1732 <= (i / fs) < 1773 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
time = pd.Series([i for i in range(3600) if i < 1732 or i >= 1773])
time = time.sample(n=int(len(x_temp2)/fs), replace=False).sort_values().multiply(fs)
for t in time:
    x_temp2 = pd.concat([x_temp2, x_temp1.iloc[t:t+256, :]])

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(55808, 24)


In [8]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_16' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1015 <= (i / fs) < 1067 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
time = pd.Series([i for i in range(3600) if i < 1015 or i >= 1067])
time = time.sample(n=int(len(x_temp2)/fs), replace=False).sort_values().multiply(fs)
for t in time:
    x_temp2 = pd.concat([x_temp2, x_temp1.iloc[t:t+256, :]])

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(82432, 24)


In [9]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_18' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1720 <= (i / fs) < 1811 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
time = pd.Series([i for i in range(3600) if i < 1720 or i >= 1811])
time = time.sample(n=int(len(x_temp2)/fs), replace=False).sort_values().multiply(fs)
for t in time:
    x_temp2 = pd.concat([x_temp2, x_temp1.iloc[t:t+256, :]])

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(128256, 24)


In [10]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_21' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 327 <= (i / fs) < 421 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
time = pd.Series([i for i in range(3600) if i < 327 or i >= 421])
time = time.sample(n=int(len(x_temp2)/fs), replace=False).sort_values().multiply(fs)
for t in time:
    x_temp2 = pd.concat([x_temp2, x_temp1.iloc[t:t+256, :]])

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(175360, 24)


In [11]:
# Import data
file_path = '/Users/bryanmcelvy/Documents/physionet.org/files/chbmit/1.0.0/chb01'
file = read_raw_edf(input_fname = file_path + '/chb01_26' + '.edf', preload=False, verbose='ERROR')
x_temp1 = pd.DataFrame(np.transpose(file.get_data()), columns = file.ch_names)

# Annotate rows (Class 0 – Normal, Class 1 – Seizure)
x_temp1.insert(loc=0, column='State', value = [1 if 1862 <= (i / fs) < 1964 else 0 for i in range(len(x_temp1))])

# Separate Class 0 and Class 1 samples
x_temp2 = pd.DataFrame(x_temp1.loc[x_temp1['State']==1, :]) # Isolate Class 1 samples
x_temp1 = x_temp1.drop(index=x_temp1.loc[x_temp1['State']==1, :].index) # Remove Class 1 samples

# Undersample Class 0 samples via random selection to create balanced class ratio
time = pd.Series([i for i in range(3600) if i < 1862 or i >= 1964])
time = time.sample(n=int(len(x_temp2)/fs), replace=False).sort_values().multiply(fs)
for t in time:
    x_temp2 = pd.concat([x_temp2, x_temp1.iloc[t:t+256, :]])

x_data = pd.concat([x_data, x_temp2]).reset_index(drop=True)
print(x_data.shape)

(218112, 24)


In [12]:
x_data.iloc[:, 1:] = x_data.iloc[:, 1:].multiply(1E6)
x_data

,State,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,...,F8-T8,T8-P8-0,P8-O2,FZ-CZ,CZ-PZ,P7-T7,T7-FT9,FT9-FT10,FT10-T8,T8-P8-1
0,1,57.240537,45.909646,55.677656,3.321123,-21.684982,42.393162,84.590965,57.631258,10.354090,...,45.518926,32.625153,-57.240537,42.783883,113.894994,-55.286935,18.949939,24.810745,6.446886,32.625153
1,1,53.333333,50.989011,56.459096,2.539683,-19.731380,45.518926,88.888889,50.207570,25.592186,...,39.267399,21.294261,-51.379731,35.750916,114.676435,-56.068376,15.824176,36.923077,9.181929,21.294261
2,1,45.909646,59.194139,54.896215,-0.976801,-23.247863,50.207570,88.888889,43.174603,38.876679,...,46.300366,10.744811,-70.134310,25.592186,113.504274,-54.505495,3.711844,54.896215,12.698413,10.744811
3,1,42.393162,64.273504,52.161172,-4.102564,-22.075702,54.505495,83.028083,40.048840,45.518926,...,50.989011,3.711844,-81.855922,20.122100,113.894994,-51.770452,-6.446886,68.962149,15.824176,3.711844
4,1,46.300366,58.021978,46.691087,-2.539683,-16.214896,53.333333,80.293040,31.062271,50.989011,...,40.830281,-5.665446,-76.776557,21.684982,115.457875,-46.300366,2.930403,73.650794,12.698413,-5.665446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218107,0,59.975580,67.008547,-12.698413,-73.650794,-14.261294,33.406593,8.791209,11.526252,-25.982906,...,13.089133,9.963370,2.930403,38.095238,-40.439560,13.089133,-36.141636,-81.465201,-16.214896,9.963370
218108,0,53.724054,71.306471,-13.870574,-73.260073,-19.731380,39.267399,7.619048,10.744811,-15.433455,...,21.294261,11.526252,-13.089133,42.783883,-46.691087,14.261294,-27.936508,-82.246642,-9.181929,11.526252
218109,0,54.114774,80.293040,-19.340659,-67.789988,-11.526252,47.081807,-0.195360,11.916972,-4.493284,...,12.698413,9.963370,-3.711844,51.770452,-46.691087,19.731380,-19.731380,-85.763126,-13.089133,9.963370
218110,0,56.849817,83.028083,-21.684982,-59.975580,-10.354090,57.240537,-2.539683,14.261294,2.539683,...,16.605617,12.698413,-0.195360,55.677656,-44.737485,22.075702,-11.135531,-82.637363,-15.042735,12.698413


In [13]:
x_data.to_csv(path_or_buf='data.csv', index=False)